Pairs of functions $f$ and $g$ from finite sets to themselves having the property that $fgf=f$ and $gfg=g$ occur in chip firing (Todo ref), rotor routing (Todo ref) and bulgarian solitaire.  Counting the number of ordered pairs $(f,g)$ that are pseudoinverses of each other is straightforward.

Let us assume wlog that our underlying set is $[n]=\{1, 2, \ldots, n\}$.

First let us count the number of pairs that satisfy the first condition $fgf=f$.  The condition only constrains $g$ on the image of $f$, so if $y=f(x)$ is in the image of $f$, then $f(g(y))=y$ and so $g(y)\in f^{-1}(y)$ for all $y$ in the image of $f$.  In particular, $g$ is injective on the image of $f$.  Let $k$ be the cardinality of the image of $f$.  There are $n!/(n-k)!$ ways of specifying $g$ on the image of $f$, as we need to specify $k$ distinct ordered values out of $[n]$.  This in turn specifies $f$ on the $k$ values of $f$, as $f(g(y)) = y$.  There are $k$ choices for each of remaining $n-k$ values of $f$, and $n$ choices of the remining values of $g$.

Combining all of these, along with the fact that there are $\binom{n}{k}$ ways of choosing the image of $f$ gives us
$$ \sum_{k=0}^n \binom{n}{k} \frac{n!}{(n-k)!} (nk)^{n-k} $$ pairs $(f,g)$ such that $fgf=f$. Including the zero index has no effect except in the $n=0$ case where the trivial function has an image of cardinality 0.

This gives us [A239768](https://oeis.org/A239768)

In [ ]:
def count_one_sided_inverse(n):
    return sum(binomial(n,k) * factorial(n) * (n*k)^(n-k) /factorial(n-k) for k in range(n+1))
print "A239768 ", [count_one_sided_inverse(n) for n in range(10)]

Counting the pairs of functions that satisfy both constraints takes only a slight modification of the above analysis.  For values $y$ in the image of $f$ we have that $f(g(y))=y$ so $g(f(g(y)))=g(y)$ so we only need to look at values outside the image of $f$.  Here if $g(x)=z$ we need $g(f(z))=z$.  This happens whenever $z = g(y)$ for some $y$ in the image of $f$.  As $g$ is injective on the image of $f$, there are $k$ choices for each value of $g(z)$ for $z$  not in the image of $f$.  So the total count becomes
$$ \sum_{k=0}^n \binom{n}{k} \frac{n!}{(n-k)!} k^{2(n-k)}. $$

This is [A277337](https://oeis.org/A277337)

In [ ]:
def count_pseudoinverse(n):
    return sum(binomial(n,k) * factorial(n) * k^(2*(n-k)) /factorial(n-k) for k in range(n+1))
print "A239768 ", [count_pseudoinverse(n) for n in range(10)]

On a finite set a pair of pseudoinverse functions $(f,g)$ has the property that $(fg)^2=fg$ and $(gf)^2 = gf$.  Is the reverse true?

In [ ]:
import itertools as it
def count_coincidences(n, test):
    count = 0
    for f in it.product(range(n), repeat=n):
        for g in it.product(range(n), repeat=n):
            good = test(n,f,g)
            if good:
                count += 1
    return count

def check_idempotent(n, f, g):
    for i in range(n):
        if f[g[i]] != f[g[f[g[i]]]] or g[f[i]] != g[f[g[f[i]]]]:
            return False
    return True

print [count_coincidences(n, check_idempotent) for n in range(1,5)]

Nope.

So let us first look at the pairs $(f,g)$ where $fg$ is idempotent.  Let there be $k$ points $y$ where $f(g(y))=y$ then $f$ maps every point in the image of $g$ to one of these points, and $f$ is arbitrary on the points not in the image of $g$.

We can divide things up

- $j$ points $z$ not fixed by $fg$ where $g(z)=g(y)$ with $y$ fixed by $fg$.
- $l$ points in the image of $g$ that are not in the image of $g(Y)$.

So $l \le j \le n-k$.  There are $n!/(n-k)!$ ways of defining $g$ on $Y$.  There are $l!\left\{{j}\atop {l}\right\}$ ways of assigning $g$ on the $j$ points and $k^{n-k-j}$ ways of assigning the other points. Finally there are $k^l$ ways of assigning $f$ on the $l$ points, and $n^{n-k-l}$ ways of assigning $f$ on the remainder.  So, combining everything we get
$$\sum_{k=1}^n \frac{n!}{(n-k)!}\binom{n}{k}\sum_{j=0}^{n-k}\binom{n-k}{j}\sum_{l=0}^j \binom{n-k}{l}
l!\left\{{j} \atop {l}\right\} k^{n-k-j}k^l n^{n-k-l}$$

expanding and cancelling

$$\sum_{k=1}^n \frac{(n!)^2}{k!}\sum_{j=0}^{n-k}\frac{1}{j!(n-k-j)!}\sum_{l=0}^j\frac{1}{(n-k-l)!}\left\{{j}\atop {l}\right\} k^{n-k-j+l} n^{n-k-l}$$

In [ ]:
def compute_idempotents(n):
    return sum(factorial(n)^2/factorial(k) * sum((1/(factorial(j)*factorial(n-k-j))) * sum((1/factorial(n-k-l)) *
                                    stirling_number2(j,l) * k^(n-k-j+l) * n^(n-k-l) 
                                        for l in range(j+1)) for j in range(n-k+1)) for k in range(1,n+1))
def one_sided_test(n,f,g):
    for i in range(n):
        if f[g[i]] != f[g[f[g[i]]]]:
            return False
    return True

#takes a long time for n=6 try something smaller if you are in a hurry
print [count_coincidences(n, one_sided_test) for n in range(1,5)]
print [compute_idempotents(n) for n in range(1,5)]

Shockingly, it works.

Now let us try to count with both constraints. 

The points $z$ where $g(f(z))=z$ are just the image under $g$ of the points $y$ where $f(g(y))=y$.  The only modification that we need to make are to restrict the values of $f$ on the set $n-k-l$ to $n-j$ distinct values that are either fixed by $gf$ or are mapped to one of those points by $g$ giving us

$$\sum_{k=1}^n \frac{(n!)^2}{k!}\sum_{j=0}^{n-k}\frac{1}{j!(n-k-j)!}\sum_{l=0}^j\frac{1}{(n-k-l)!}\left\{{j}\atop {l}\right\} k^{n-k-j+l} {(n-j)}^{n-k-l}$$

In [ ]:
def compute_double_idempotents(n):
    return sum(factorial(n)^2/factorial(k) * sum((1/(factorial(j)*factorial(n-k-j))) * sum((1/factorial(n-k-l)) *
                                    stirling_number2(j,l) * (k)^(n-k-j+l) * (n-j)^(n-k-l) 
                                        for l in range(j+1)) for j in range(n-k+1)) for k in range(1,n+1))

print [count_coincidences(n, check_idempotent) for n in range(1,5)]
print [compute_double_idempotents(n) for n in range(1,10)]

Shockingly, this works as well. There should be some way of simplifying the idempotent counting function.

TODO:

+ compute asymtotic approximations for the counts.
+ See if there is some way of fitting this into the Moore-Penrose framework.
+ Look at further restrictions -- e.g. $f$ and $g$ have the same limit cycles.
+ Look at when f and g are conjugate - $g = a^{-1}fa$ for some permutation or involution $a$.